In [7]:
import sklearn 
import numpy as np
import pandas as pd
import sklearn.preprocessing as pre

In [50]:
data_train= pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data_test.loc[ (data_test.Fare.isnull()), 'Fare' ] = 0


In [22]:
def set_Cabin_type(df):
    df.loc[ (df.Cabin.isnull()), 'Cabin' ] = "No"
    df.loc[ (df.Cabin.notnull()), 'Cabin' ] = "Yes"
    return df

In [23]:
from sklearn.ensemble import RandomForestRegressor

def set_missing_age(df):
    age_df = df[['Age','Pclass','SibSp','Parch','Fare']]#.astype(np.float)
    #print(type(age_df))
    train_age = age_df[age_df['Age'].notnull()].as_matrix()#.astype(np.float)
    test_age= age_df[age_df['Age'].isnull()].as_matrix()#.astype(np.float)
    age = train_age[:,0]
    feature = train_age[:,1:]
    #print(test_age,train_age)
    rfr = RandomForestRegressor(random_state=0,n_estimators=2000,n_jobs=-1)
    rfr.fit(feature, age)
    result = rfr.predict(test_age[:,1:])
    #print(result)
    df1=df.copy()
    df1.loc[df['Age'].isnull(),'Age']= result
    
    #print(type(df1))
    return df1,rfr

In [38]:
def get_dummy(da_tr):
    dummy_cabin=pd.get_dummies(da_tr['Cabin'],prefix='Cabin')
    dummy_sex = pd.get_dummies(da_tr['Sex'],prefix='Sex')
    dummy_embarked=pd.get_dummies(da_tr['Embarked'],prefix='Embarked')
    dummy_pclass =pd.get_dummies(da_tr['Pclass'],prefix='Pclass')
    dummy_Sex_Pclass = pd.get_dummies(da_tr['Sex_Pclass'], prefix= 'Sex_Pclass')

    da_tr=pd.concat([da_tr,dummy_cabin,dummy_embarked,dummy_pclass,dummy_sex,dummy_Sex_Pclass],axis=1) # do not forget axis
    da_tr.drop(['Pclass','Ticket','Embarked','Cabin','Name','Sex','Sex_Pclass'],axis=1,inplace=True)
    return da_tr

In [39]:
def set_sex_pclass(df):
    df['Sex_Pclass']=df.Sex +"_" + df.Pclass.map(str)
    return df

In [40]:
def feature_scaler(df):
    fs=pre.StandardScaler()
    age_fs = fs.fit(df['Age'].values.reshape(-1,1))
    df['Age_fs']=fs.fit_transform(df['Age'].values.reshape(-1,1),age_fs)
    fare_fs = fs.fit(df['Fare'].values.reshape(-1,1))
    df['Fare_fs']=fs.fit_transform(df['Fare'].values.reshape(-1,1),fare_fs)
    #df.drop(['Age','Fare'],axis=1,inplace=True)
    return df

In [41]:
from sklearn import linear_model
df =data_train.copy()
df =set_Cabin_type(df)
df,rfr =set_missing_age(df)
df = feature_scaler(df)
df = set_sex_pclass(df)
df = get_dummy(df)
train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass.*')
train_np = train_df.as_matrix()
x = train_np[:,1:]
y = train_np[:,0]

In [42]:
clf = linear_model.LogisticRegression(C=1.0,penalty='l1',tol=1e-6)
clf.fit(x,y)
clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [54]:
df_te = data_test.copy()
df_te = set_Cabin_type(df_te)
df_te,rfr_te = set_missing_age(df_te)
df_te = feature_scaler(df_te)
df_te = set_sex_pclass(df_te)
df_te = get_dummy(df_te)

In [57]:
test_df = df_te.filter(regex='Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass.*')

In [58]:
pre = clf.predict(test_df)

In [62]:
result=pd.DataFrame({'PassengerId':df_te['PassengerId'].as_matrix(),"Survived":pre.astype(np.int32)})

In [63]:
result.to_csv('pre.csv',index =False)

In [65]:
from sklearn.ensemble import BaggingRegressor
bagging_clf = BaggingRegressor(clf, n_estimators=10, max_samples=0.8, max_features=1.0, bootstrap=True, bootstrap_features=False, n_jobs=-1)
bagging_clf.fit(x,y)
pre1 = clf.predict(test_df)
result1=pd.DataFrame({'PassengerId':df_te['PassengerId'].as_matrix(),"Survived":pre1.astype(np.int32)})
result1.to_csv('pre1.csv',index =False)

In [67]:
test_df.head(10)

,SibSp,Parch,Age_fs,Fare_fs,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Sex_Pclass_female_1,Sex_Pclass_female_2,Sex_Pclass_female_3,Sex_Pclass_male_1,Sex_Pclass_male_2,Sex_Pclass_male_3
0,0,0,0.367117,-0.496637,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1
1,1,0,1.331258,-0.511497,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0
2,0,0,2.488226,-0.463335,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0
3,0,0,-0.211367,-0.481704,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1
4,1,1,-0.597023,-0.416740,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0
5,0,0,-1.214073,-0.471623,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1
6,0,0,0.020027,-0.500221,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0
7,1,1,-0.288498,-0.117238,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0
8,0,0,-0.905548,-0.507390,1,1,0,0,0,0,1,1,0,0,0,1,0,0,0
9,2,0,-0.674154,-0.204154,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1


In [92]:
import numpy as np
import pandas as pd
from pandas import  DataFrame
from patsy import dmatrices
import string
from operator import itemgetter
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split,StratifiedShuffleSplit,StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.externals import joblib

##Read configuration parameters

train_file="train.csv"
MODEL_PATH="./"
test_file="test.csv"
SUBMISSION_PATH="./"
seed= 0

print (train_file,seed)

# 输出得分
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

#清理和处理数据
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str.find(big_string, substring) != -1:
            return substring
    print (big_string)
    return np.nan

le = preprocessing.LabelEncoder()
enc=preprocessing.OneHotEncoder()

def clean_and_munge_data(df):
    #处理缺省值
    df.Fare = df.Fare.map(lambda x: np.nan if x==0 else x)
    #处理一下名字，生成Title字段
    title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                'Don', 'Jonkheer']
    df['Title']=df['Name'].map(lambda x: substrings_in_string(x, title_list))

    #处理特殊的称呼，全处理成mr, mrs, miss, master
    def replace_titles(x):
        title=x['Title']
        if title in ['Mr','Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
            return 'Mr'
        elif title in ['Master']:
            return 'Master'
        elif title in ['Countess', 'Mme','Mrs']:
            return 'Mrs'
        elif title in ['Mlle', 'Ms','Miss']:
            return 'Miss'
        elif title =='Dr':
            if x['Sex']=='Male':
                return 'Mr'
            else:
                return 'Mrs'
        elif title =='':
            if x['Sex']=='Male':
                return 'Master'
            else:
                return 'Miss'
        else:
            return title

    df['Title']=df.apply(replace_titles, axis=1)

    #看看家族是否够大，咳咳
    df['Family_Size']=df['SibSp']+df['Parch']
    df['Family']=df['SibSp']*df['Parch']


    df.loc[ (df.Fare.isnull())&(df.Pclass==1),'Fare'] =np.median(df[df['Pclass'] == 1]['Fare'].dropna())
    df.loc[ (df.Fare.isnull())&(df.Pclass==2),'Fare'] =np.median( df[df['Pclass'] == 2]['Fare'].dropna())
    df.loc[ (df.Fare.isnull())&(df.Pclass==3),'Fare'] = np.median(df[df['Pclass'] == 3]['Fare'].dropna())

    df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

    df['AgeFill']=df['Age']
    mean_ages = np.zeros(4)
    mean_ages[0]=np.average(df[df['Title'] == 'Miss']['Age'].dropna())
    mean_ages[1]=np.average(df[df['Title'] == 'Mrs']['Age'].dropna())
    mean_ages[2]=np.average(df[df['Title'] == 'Mr']['Age'].dropna())
    mean_ages[3]=np.average(df[df['Title'] == 'Master']['Age'].dropna())
    df.loc[ (df.Age.isnull()) & (df.Title == 'Miss') ,'AgeFill'] = mean_ages[0]
    df.loc[ (df.Age.isnull()) & (df.Title == 'Mrs') ,'AgeFill'] = mean_ages[1]
    df.loc[ (df.Age.isnull()) & (df.Title == 'Mr') ,'AgeFill'] = mean_ages[2]
    df.loc[ (df.Age.isnull()) & (df.Title == 'Master') ,'AgeFill'] = mean_ages[3]

    df['AgeCat']=df['AgeFill']
    df.loc[ (df.AgeFill<=10) ,'AgeCat'] = 'child'
    df.loc[ (df.AgeFill>60),'AgeCat'] = 'aged'
    df.loc[ (df.AgeFill>10) & (df.AgeFill <=30) ,'AgeCat'] = 'adult'
    df.loc[ (df.AgeFill>30) & (df.AgeFill <=60) ,'AgeCat'] = 'senior'

    df.Embarked = df.Embarked.fillna('S')


    df.loc[ df.Cabin.isnull()==False,'Cabin'] = 1.5
    df.loc[ df.Cabin.isnull()==True,'Cabin'] = 0.5

    df['Fare_Per_Person']=df['Fare']/(df['Family_Size']+1)

    #Age times class

    df['AgeClass']=df['AgeFill']*df['Pclass']
    df['ClassFare']=df['Pclass']*df['Fare_Per_Person']


    df['HighLow']=df['Pclass']
    df.loc[ (df.Fare_Per_Person<8) ,'HighLow'] = 'Low'
    df.loc[ (df.Fare_Per_Person>=8) ,'HighLow'] = 'High'



    le.fit(df['Sex'] )
    x_sex=le.transform(df['Sex'])
    df['Sex']=x_sex.astype(np.float)

    le.fit( df['Ticket'])
    x_Ticket=le.transform( df['Ticket'])
    df['Ticket']=x_Ticket.astype(np.float)

    le.fit(df['Title'])
    x_title=le.transform(df['Title'])
    df['Title'] =x_title.astype(np.float)

    le.fit(df['HighLow'])
    x_hl=le.transform(df['HighLow'])
    df['HighLow']=x_hl.astype(np.float)


    le.fit(df['AgeCat'])
    x_age=le.transform(df['AgeCat'])
    df['AgeCat'] =x_age.astype(np.float)

    le.fit(df['Embarked'])
    x_emb=le.transform(df['Embarked'])
    df['Embarked']=x_emb.astype(np.float)

    df = df.drop(['PassengerId','Name','Age','Cabin'], axis=1) #remove Name,Age and PassengerId


    return df

#读取数据
traindf=pd.read_csv(train_file)
##清洗数据
df=clean_and_munge_data(traindf)
########################################formula################################
 
formula_ml='Survived~Pclass+C(Title)+Sex+C(AgeCat)+Fare_Per_Person+Fare+Family_Size' 

y_train, x_train = dmatrices(formula_ml, data=df, return_type='dataframe')
y_train = np.asarray(y_train).ravel()
#print (y_train.shape,x_train.shape

##选择训练和测试集
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2,random_state=seed)
#初始化分类器
clf=RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=5, min_samples_split=2,
  min_samples_leaf=1, max_features='auto',    bootstrap=False, oob_score=False, n_jobs=1, random_state=seed,
  verbose=0)

###grid search找到最好的参数
param_grid = dict( )
##创建分类pipeline
pipeline=Pipeline([ ('clf',clf) ])
grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=3,scoring='accuracy',\
cv=StratifiedShuffleSplit(Y_train, n_iter=10, test_size=0.2, train_size=None, \
random_state=seed)).fit(X_train, Y_train)
# 对结果打分
print("Best score: %0.3f" % grid_search.best_score_)
print("best ",grid_search.best_estimator_)

report(grid_search.grid_scores_)
 
print('-----grid search end------------')
print ('on all train set')
scores = cross_val_score(grid_search.best_estimator_, x_train, y_train,cv=3,scoring='accuracy')
print (scores.mean(),scores)
print ('on test set')
scores = cross_val_score(grid_search.best_estimator_, X_test, Y_test,cv=3,scoring='accuracy')
print (scores.mean(),scores)

# 对结果打分

print(classification_report(Y_train, grid_search.best_estimator_.predict(X_train) ))
print('test data')
print(classification_report(Y_test, grid_search.best_estimator_.predict(X_test) ))

model_file=MODEL_PATH+'model-rf.pkl'
joblib.dump(grid_search.best_estimator_, model_file)

train.csv 0
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV]  ................................................................
[CV] ....................................... , score=0.860140 -   0.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] ....................................... , score=0.832168 -   0.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV] ....................................... , score=0.818182 -   0.6s
[CV]  ................................................................
[CV] ....................................... , score=0.832168 -   0.6s
[CV]  ................................................................
[CV] ....................................... , score=0.811189 -   0.6s
[CV]  ................................................................
[CV] ....................................... , score=0.874126 -   0.6s
[CV]  ................................................................
[CV] ....................................... , score=0.811189 -   0.6s
[CV]  ................................................................
[CV] ....................................... , score=0.783217 -   0.6s
[CV]  ................................................................
[CV] ....................................... , score=0.825175 -   0.6s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.7s finished


Best score: 0.829
best  Pipeline(memory=None,
     steps=[('clf', RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False))])
Model with rank: 1
Mean validation score: 0.829 (std: 0.024)
Parameters: {}

-----grid search end------------
on all train set
0.824915824916 [ 0.81144781  0.83164983  0.83164983]
on test set
0.782203389831 [ 0.76666667  0.78333333  0.79661017]
             precision    recall  f1-score   support

        0.0       0.86      0.90      0.88       439
        1.0       0.83      0.75      0.79       273

avg / total       0.85      0.85      0.85       712

test data
             precision    recall  f1-scor

['./model-rf.pkl']

In [81]:
grid_search.best_estimator_

Pipeline(memory=None,
     steps=[('clf', RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False))])

In [109]:
testdf=pd.read_csv(test_file)
testdf_clean =clean_and_munge_data(testdf)
test_clean = testdf_clean['Pclass','C(Title)+Sex+C(AgeCat)+Fare_Per_Person+Fare+Family_Size']

In [119]:
# testdf_clean.info()
#x_train.info()
formula_te='Embarked~Pclass+C(Title)+Sex+C(AgeCat)+Fare_Per_Person+Fare+Family_Size' 

_,x_test = dmatrices(formula_te, data=testdf_clean, return_type='dataframe')

In [123]:
estimator = joblib.load(model_file)
pre3=estimator.predict(x_test)

In [132]:
result = pd.DataFrame({"PassengerId":data_test["PassengerId"].as_matrix(),"Survived":pre3.astype(np.int32)})
result.to_csv('pre3.csv',index=False)